<a href="https://colab.research.google.com/github/Fly143/ikatago-for-colab/blob/main/ikatago_for_colab_ipynb%22%E5%A2%A8%E7%99%BD%E4%BF%AE%E5%A4%8D%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 如何运行？
1). 点击安装代码里的运行单元格,安装完成后进行下一步.  
2). 修改运行代码里的USER_NAME, USER_PASSWORD, 点击运行单元格即可.  
3). 点这里下载ikatago客户端 (1.6.0版本):  
https://github.com/kinfkong/ikatago-client/releases/tag/v1.6.0

ikatago连接参数样例:
```
ikatago.exe --platform colab --username xxxxx --password xxxxx
```
username及password要改成你自己的值.



human权重对弈方法

ikatago.exe --platform colab --username xxxxx --password xxxxx --kata-config human5k --kata-override-config humanSLProfile=preaz_5k -- gtp -human-model /content/work/data/weights/human.bin.gz

建议k级用human5k，d级用human9d，把--kata-config human5k改为--kata-config human9d

humanSLProfile=preaz_5k为可选参数，用于指定棋力，更改风格等
可用选项包括：
- preaz_{RANK from 20k to 9d}：模仿特定段位的玩家，在AlphaZero开局风格流行之前的风格。
- rank_{RANK from 20k to 9d}：模仿特定段位的玩家，在人类开局因AlphaZero影响而变化之后的风格。
- preaz_{BRANK}_{WRANK} 或 rank_{BRANK}_{WRANK}：同上，但分别模仿黑方为BR段和白方为WR段的玩家对弈时的行为，考虑到对手的实力较强或较弱。
警告：对于段位差异大于9段的情况，或者与游戏中使用的让子数严重不匹配的情况下，由于缺乏训练数据，模型的表现可能不佳！请谨慎实验。
- proyear_{YEAR from 1800 to 2023}：模仿特定年份的历史职业棋手或院生的风格。

### 更多用法可以看下面FAQ部分

# 安装

In [ ]:
# Optional Args
# Supports: 18b, 20b, 28b，40b, 60b, (或b60, b40...除28b外，其余需要指定权重)
# Notes:
# 1. 选择 28b 会自动使用elo最强的权重
# 2. 如果要指定kata1权重，写: 60b578, 40b1110 18b583 或 b60s578, b40s1110, kata1-b60-s578
#   换言之，只要是kata1的权重，格式很灵活，随便写都可以
#   指定需要加载的权重请在ikatago命令行添加例如: --kata-weight 40b （不加默认使用28b)
WEIGHT_FILE="28b"
# Supports:CUDA, TRT
# Notes:
# 1. 指定需要加载的引擎请在ikatago命令行添加例如: --kata-name katago-TENSORRT (不加默使用你选择的)
#   但注意TENSORRT后端需要安装依赖，要使用请至少 WEIGHT_FILE="ALL" 或 KATAGO_BACKEND="TRT" 有其中一个
# 2. Colab上分配的卡跑KataGO, CUDA 和 TENSORRT 相比较，TENSORRT提升约10%-20%，CUDA加载更快
# 3.TENSORRT引擎。加载初始化大概需要1~2分钟！cuda的话十几秒！
KATAGO_BACKEND="CUDA"

%cd /content

# install
!wget -q https://github.com/Fly143/ikatago-for-colab/releases/download/1.14.1/libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb -O libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
!dpkg -i libssl1.1_1.1.1f-1ubuntu2.17_amd64.deb
!wget -q https://github.com/Fly143/ikatago-for-colab/releases/download/1.15.3/install.sh -O install.sh
!chmod +x ./install.sh
!./install.sh $WEIGHT_FILE $KATAGO_BACKEND
%cd /content/work/data/weights
!wget -q https://github.com/lightvector/KataGo/releases/download/v1.15.0/b18c384nbt-humanv0.bin.gz -O human.bin.gz

### 根据kinfkong大佬脚本修改 ###

# 运行

In [ ]:
# Notes: 请自行修改用户名和密码
USER_NAME="qwert"
USER_PASSWORD="12345678"

FRPC="""
### YOUR FRPC CONTENT ###

[common]
tls_enable = true
server_addr = {{ .Envs.KNAT_SERVER_ADDR }}
server_port = {{ .Envs.KNAT_SERVER_PORT }}
token = {{ .Envs.KNAT_SERVER_TOKEN }}

[kinfkong-ssh]
type = tcp
local_ip = 127.0.0.1
local_port = 2222
remote_port = 0

### END YOUR FRPC CONTENT ###
"""

%cd /content/work

# change the frp
!echo """$FRPC""" > config/frpc.txt
!chmod +x ./change-frpc.sh
!./change-frpc.sh $USER_NAME

# run the server
!chmod +x ./run.sh
!./run.sh $USER_NAME $USER_PASSWORD

### 根据kinfkong大佬脚本修改 ###

# FAQ

### 1. 如何换frp （非必需，但换到合适的frp会更快)
1). 在第三方frp提供商，比如https://natfrp.com  中创建TCP隧道  
2). 隧道设为本地ip 127.0.0.1, 本地端口: 2222  
3). 找到frp提供商给你的配置文件，复制  
4). 粘贴在下面代码的 `### YOUR FRPC CONTENT ###` 与 `### END YOUR FRPC CONTENT  ###` 之间即可。

### 2. 如何用Sabaki, Lizzie等客户端连接colab上的katago?

#### Lizzie用法
```
<ikatago程序路径> --platform colab --username <你设置的用户名> --password <你设置的密码>
```
比如，Windows下可能是这样子:
```
C:\xxx\ikatago.exe --platform colab --username xxxxx --password xxxxx
```

#### Sabaki的用法
有三行，  
第一行: 引擎名字，随便起一个名字  
第二行: 程序路径，就是ikatago在你本机的路径，比如, C:\xxx\ikatago.exe  
第三行: 运行参数: --platform colab --username <你设置的用户名>   --password <你设置的密码>

### 3. 如何用SSH连接？
建议使用ikatago客户端进行连接，不建议直接用ssh. 但用ssh也是可以的。  
上面的代码运行后，输出里，你可以找到:  
SSH_HOST ssh主机  
SSH_PORT ssh端口  
ssh用户名就是你设定的用户名  
ssh密码就是你设定的密码  
ssh运行命令是"run-katago"

### 4. 如何修改katago配置文件？
你可以通过ikatago客户端，通过`--kata-local-config`来指定你自己本机上的配置文件，比如:

```
ikatago.exe --kata-local-config C:\xxx.cfg --username xxx ...
```

### 5. 如何使用colab内置配置文件?
你可以通过ikatago客户端，通过`--kata-config`来指定colab上内置的配置文件，比如:

```
ikatago.exe --kata-config dynamic_pda --username xxx ...
```
目前colab上内置了如下的配置文件: (感谢@LK @BionicMed 等群友的整理)
```
default_gtp  dynamic_pda  2stones_handicap  3stones_handicap  4stones_handicap  5stones_handicap  6stones_handicap  7+stones_handicap  
```
**注: 请自行注要区别`--kata-config`与`--kata-local-config`的联系与区别。如果两个同时指定，优先用kata-local-config


### 6. 如何运行katago benchmark, genconfig等命令 (高级用法，一般人可以忽略)

ikatago客户端（要升级到1.3.1或以上)，支持所有katago的子命令，把整条原来katago的命令copy到最后，然后用"--"隔开就行
比如，运行benchmark:

```
ikatago.exe --kata-local-config C:\xxx.cfg --kata-weight 20b -- benchmark -v xx -t x,x...(v指visits，以5000为宜，t指线程数，后填需测试的线程数用逗号隔开。
```
再比如gtp
```
ikatago.exe --kata-local-config C:\xxx.cfg --kata-weight 20b -- gtp -override-config maxVisits=1000
```
**注: 原来的子命令里如果有"-model"或"-config"参数的，你也可以照常指定，不指定的话，就分别会用`--kata-weight`、`--kata-config`(或`--kata-local-config`)里的。

### 7. 为什么分析时选点最多就20个?
因为为了大量降低数据传输量，Lizzie等软件，默认也是只显示前10个，所以不影响体验。如果要增加显示选点的个数（或不限制），可以:  
    "--transmit-move-num 10" 这样子来修改.  
    "--transmit-move-num 0" 表示不限制.


# 以下的都是调试用的代码，普通用户可以忽略下面的所有内容

显示GPU类型

In [ ]:
!nvidia-smi